### Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


#Вариант 1: Парсинг ПОСЛЕ регистрозависимого поиска на странице ключевых слов KEYWORDS.
#KEYWORDS расширен на случай отсутстсвия ключевых слов в ленте
KEYWORDS = ['python', 'парсинг']
req = requests.get('https://habr.com/ru/all/')
news_list = []
news = BeautifulSoup(req.text, 'html.parser')
for el in news.find_all("div", attrs={"class": "tm-article-snippet"}):
    for keyword in KEYWORDS:
        if keyword in el.text:
            news_dict = {}
            news_dict['дата'] = el.find('time').get('title')
            news_dict['заголовок'] = el.find("a", attrs = {"class" : "tm-article-snippet__title-link"}).text
            news_dict['ссылка'] = 'https://habr.com' + el.find("a", attrs = {"class" : "tm-article-snippet__title-link"}).get('href')
            news_list.append(news_dict)
news_df = pd.DataFrame(news_list)
news_df.head()

,дата,заголовок,ссылка
0,"2021-07-22, 21:22",Классификатор обращений пользователей (1C + py...,https://habr.com/ru/post/569206/
1,"2021-07-22, 16:49",[Личный опыт] Эстония: почему сюда нанимают мн...,https://habr.com/ru/company/gms/blog/569152/


In [2]:
#Вариант 2: Парсинг данных c последующей проверкой превью и основного текста на ключевые слова,
#и если ключевые слова находятся, то данные добавляются в список.
KEYWORDS = ['python', 'парсинг', 'Desktop']
req = requests.get('https://habr.com/ru/all/')
full_news_list = []
full_news = BeautifulSoup(req.text, 'html.parser')
for el in full_news.find_all("div", attrs={"class": "tm-article-snippet"}):
    full_news_dict = {}
    full_news_dict['дата'] = el.find('time').get('title')
    full_news_dict['заголовок'] = el.find("a", attrs = {"class" : "tm-article-snippet__title-link"}).text
    full_news_dict['ссылка'] = 'https://habr.com' + el.find("a", attrs = {"class" : "tm-article-snippet__title-link"}).get('href')
    link_req = requests.get(full_news_dict['ссылка'])
    full_news_dict['текст_статьи'] = BeautifulSoup(link_req.text, 'html.parser').find("div", attrs = {"xmlns" : "http://www.w3.org/1999/xhtml"}).text #div xmlns="http://www.w3.org/1999/xhtml"
    for keyword in KEYWORDS:
        if (keyword in full_news_dict['текст_статьи']) or (keyword in el.text):
            full_news_list.append(full_news_dict)
full_news_df = pd.DataFrame(full_news_list)
full_news_df.head()

,дата,заголовок,ссылка,текст_статьи
0,"2021-07-22, 21:22",Классификатор обращений пользователей (1C + py...,https://habr.com/ru/post/569206/,1. Описание задачиВ нашей компании очень много...
1,"2021-07-22, 18:17",Разбор конкурса IDS Bypass на Positive Hack Da...,https://habr.com/ru/company/pt/blog/569168/,Уже второй раз на конференции Positive Hack Da...
2,"2021-07-22, 16:49",[Личный опыт] Эстония: почему сюда нанимают мн...,https://habr.com/ru/company/gms/blog/569152/,Эстония не попадает в топы стран для эмиграции...


#### Почему не добавляю строки сразу в датафрейм
В данной задаче не существенно, но при работе с бОльшим количеством данных, опираясь на строки из документации по Pandas:

*Iteratively appending rows to a DataFrame can be more computationally intensive than a single concatenate. A better solution is to append those rows to a list and then concatenate the list with the original DataFrame all at once. (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html)*

решено не добавлять данные в датафрейм построчно, а сформировать его по итоговому списку.

https://stackoverflow.com/a/62734983/12107593

### Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [3]:
EMAIL = ["xxx@x.ru", "yyy@y.com", "xxx@ya.ru"]
URL = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
data = {
    "emailAddresses" : EMAIL
}
headers = {
    "Vaar-Header-App-Build-Version": "1.0.0",
    "Vaar-Header-App-Product" : "hackcheck-web-avast",
    "Vaar-Header-App-Product-Name" : "hackcheck-web-avast",
    "Vaar-Version" : "0"
}
res = requests.post(URL, headers=headers, json=data)
res

<Response [200]>

In [5]:
#Формируем DataFrame с источниками утечек
breaches_list = []
df_breaches = pd.DataFrame(res.json()['breaches'].values())
df_breaches.head()

,breachId,site,recordsCount,description,publishDate,statistics
0,2,linkedin.com,158591429,"In 2012, online professional networking platfo...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 111975337, 'emai..."
1,3,adobe.com,152046506,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 129430596, 'emai..."
2,12,vk.com,110121799,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z,"{'usernames': 0, 'passwords': 107425364, 'emai..."
3,15,imesh.com,50913222,"In June 2016, a cache of over 51 million user ...",2016-10-23T00:00:00Z,"{'usernames': 50909015, 'passwords': 50913222,..."
4,41,dropbox.com,68591031,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z,"{'usernames': 0, 'passwords': 68591031, 'email..."


In [6]:
#df_response - DataFrame из email и breachId
df_response = pd.concat({k: pd.Series(list(v.values())[0]) for k, v in res.json()['summary'].items()})
df_response = df_response.reset_index().drop(columns=['level_1']).rename(columns={"level_0": "email", 0: "breachId"})
#Объединение для расшифровки источника
df_response.merge(df_breaches.drop(columns=['recordsCount', 'statistics']), how = 'left', on = 'breachId')


,email,breachId,site,description,publishDate
0,xxx@x.ru,3,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z
1,xxx@x.ru,12,vk.com,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z
2,xxx@x.ru,15,imesh.com,"In June 2016, a cache of over 51 million user ...",2016-10-23T00:00:00Z
3,xxx@x.ru,2961,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da...",2017-01-31T00:00:00Z
4,xxx@x.ru,3164,cfire.mail.ru,"In July and August of 2016, two criminals carr...",2017-02-14T00:00:00Z
5,xxx@x.ru,3176,parapa.mail.ru,"In July and August 2016, two criminals execute...",2017-02-14T00:00:00Z
6,xxx@ya.ru,12,vk.com,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z
7,xxx@ya.ru,41,dropbox.com,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z
8,xxx@ya.ru,60,qip.ru,"In 2011, Russian instant messaging service pro...",2016-11-01T00:00:00Z
9,xxx@ya.ru,88,ir.netease.com,"In October 2015, the Chinese internet and gami...",2016-11-07T00:00:00Z


Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.

Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [8]:
from yaml import full_load

GROUP = 'netology'
with open('config.yaml', mode = 'r', encoding = 'utf-8') as f:
    config = full_load(f)
TOKEN = config['access_token']
URL = 'https://api.vk.com/method/wall.get'

params = {
    'domain': GROUP,
    'count': 50,
    'offset': 0,
    'access_token': TOKEN,
    'v': 5.84
}
r = requests.get(URL, params = params)
r.json()

{'response': {'count': 7796,
  'items': [{'id': 92992,
    'from_id': -30159897,
    'owner_id': -30159897,
    'date': 1625668202,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Срочное сообщение от коллеги в пятницу вечером, голосовое в рабочем чате, эмодзи по поводу и без 😵 Собрали вредные советы по деловому общению в сети. А что вас раздражает в переписках?\n\nА о том, как нужно общаться с коллегами в сети — читайте в Медиа → http://netolo.gy/gLU',
    'is_pinned': 1,
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -7,
       'date': 1625666729,
       'id': 457250339,
       'owner_id': -30159897,
       'has_tags': False,
       'access_key': '471345854bccd83887',
       'sizes': [{'height': 130,
         'url': 'https://sun9-22.userapi.com/impg/iectCridpvWhUNEoR5zVaSyyOFdR_waogFdx_g/qwvAPPP3Yhs.jpg?size=130x130&quality=96&sign=6f18673c1b2998e9548fa377b3bb07bc&c_uniq_tag=nYxQ90zVj302ZODOV8UwTV1GpMzuXUr3tosXP-7IrKQ&type=album',
         'type': 'm'

In [9]:
data = [(x['date'], x['text']) for x in r.json()['response']['items']]
df_data = pd.DataFrame.from_records(data, columns=['Дата', 'Текст'])
df_data['Дата'] = pd.to_datetime(df_data['Дата'], unit='s')
df_data.head()

,Дата,Текст
0,2021-07-07 14:30:02,Срочное сообщение от коллеги в пятницу вечером...
1,2021-07-22 15:53:00,"Часто люди думают, что другие мыслят так же, к..."
2,2021-07-22 08:31:00,Доставайте попкорн — вышел новый ролик ЭтоИзи ...
3,2021-07-21 14:30:01,"Даже если вы не дизайнер или иллюстратор, важн..."
4,2021-07-21 07:13:00,Дата-сайентисты зарабатывают в среднем в два р...
